In [1]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

# import hack to allow this example file to load the SMD package without installing it
import sys
sys.path.append("SMD/")

import smd as SMD

## Setup model

In [2]:
from types import SimpleNamespace
class Model():
    def __init__(self):
        
        self.par = SimpleNamespace()
        par = self.par
        par.a = 1
        par.b = 2
        
        par.Nsim = 1000
        
        self.sol = SimpleNamespace()
        self.sim = SimpleNamespace()
        
    def update_par(self,theta,names):
        for name,val in zip(names,theta):
            setattr(self.par,name,val)
    
    def solve(self):
        self.sol.y = lambda x: self.par.a + self.par.b * x
        
    def setup_simulation(self,seed=2021):
        np.random.seed(seed)
        self.sim.x_draws = np.random.normal(size=self.par.Nsim)
        self.sim.e_draws = 0.1*np.random.normal(size=self.par.Nsim)
        
    def simulate(self):
        
        self.sim.x = self.sim.x_draws
        self.sim.y = self.sol.y(self.sim.x) + self.sim.e_draws
        
    def calc_moments(self):
        moms = OrderedDict()
        moms['avg'] = np.mean(self.sim.y)
        moms['cov'] = np.cov(self.sim.y,self.sim.x)[0,1]
        
        return moms
        

## Simulate synthetic data

In [3]:
data = Model()
data.solve()
data.setup_simulation(seed=2021)
data.simulate()

moms = data.calc_moments()
moms

OrderedDict([('avg', 1.0045484831112237), ('cov', 2.0317467972390584)])

## setup model and estimate $a$ and $b$

In [4]:
model = Model()
model.par.Nsim = 5000
model.setup_simulation(seed=2020)

In [5]:
smd = SMD.SMDClass()
# skip loading data and just insert datamoms for now
smd.datamoms = moms

In [6]:
# Estimation setup
est_par = {
    'a': {'init':0.5,'bounds':[0.0 , 2.0]},
    'b': {'init':2.5,'bounds':[0.0 , 3.0]},
}

res = smd.estimate(model,est_par,print_progress=True)

Running optimization:
   0:
     a = 0.5000
     b = 2.5000

   Number 1 in solver sequence of length 2

    obj = 0.0328, 0.0 secs, 2 func. evals
  10:
     a = 0.8187
     b = 2.1250

   Number 2 in solver sequence of length 2

    obj = 0.0000, 0.0 secs, 9 func. evals
  20:
     a = 0.9912
     b = 2.0687
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 15
         Gradient evaluations: 5


In [7]:
res.x

array([0.9914226 , 2.06864611])